In [ ]:
import numpy as np
import pandas as pd
import Algorithms.baselines as base
import Algorithms.context_knn as cknn
import Algorithms.SeqContextKNN as scknn
import Algorithms.gru4rec as gru4rec
import Algorithms.svmknn as svmknn
import time
import pickle
import argparse
import os

In [ ]:
def trainBPR(lambda_session,lambda_item):
    algo = base.BPR(lambda_session = lambda_session,lambda_item = lambda_item)

    print("Training algorithm: BPR with lambda_session {} and lambda_item {}".format(lambda_session,lambda_item))

    algo.fit(train)

    # save the model to disk
    filename = "models/valid/cb12_BPR_ls"  + str(lambda_session) + 'li'+ str(lambda_item) + ".model"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    print("Finished training. Storing model to: " + filename)
    with open(filename,'wb') as f:
        pickle.dump(algo,f)
    pass

In [ ]:
def trainIKNN(lmbd,alpha):
    algo = base.ItemKNN(lmbd=lmbd, alpha=alpha)

    print("Training algorithm: ItemKNN with lambda {} and alpha {}".format(lmbd,alpha))

    algo.fit(train)

    # save the model to disk
    filename = "models/valid/cb12_IKNN_lmbd" + str(lmbd) + 'alpha' + str(alpha) + ".model"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    print("Finished training. Storing model to: " + filename)
    with open(filename,'wb') as f:
        pickle.dump(algo,f)
    pass

In [ ]:
def trainSKNN(sampling, similarity, pop_boost):
    """
    Also known as context knn or cknn
    k = 500
    sampling = 1000
    """
    algo = cknn.ContextKNN(k = 500, sampling=sampling, similarity = similarity, pop_boost = pop_boost)

    print("Training algorithm: SKNN with sampling {}, similarity {} and pop_boost {}".format(sampling, similarity,
                                                                                                pop_boost))

    algo.fit(train)

    # save the model to disk
    filename = "models/valid/cb12_SKNN_Smpl" + str(sampling) + 'Sim' + str(similarity) + 'Pop_boost' + str(pop_boost) + ".model"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    print("Finished training. Storing model to: " + filename)
    with open(filename,'wb') as f:
        pickle.dump(algo,f)
    pass

In [ ]:
def trainSsKNN(sampling,similarity,pop_boost):

    """
    Also known as sequental context KNN
    k = 500
    sampling = 1000
    """

    algo = scknn.SeqContextKNN(sampling=sampling, similarity = similarity, pop_boost = pop_boost)

    print("Training algorithm: S-sKNN with sampling {}, similarity {} and pop_boost {}".format(sampling, similarity,
                                                                                                pop_boost))

    algo.fit(train)

    # save the model to disk
    filename = "models/valid/cb12_S-sKNN_Smpl" + str(sampling) + 'Sim' + str(similarity) + 'Pop_boost' + str(pop_boost) + ".model"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    print("Finished training. Storing model to: " + filename)
    with open(filename,'wb') as f:
        pickle.dump(algo,f)
    pass

In [ ]:
def trainSVMKNN(sampling,similarity,pop_boost):
    """
    k = 500
    sampling = 1000
    """

    algo = svmknn.VMContextKNN(sampling=sampling, similarity = similarity, pop_boost = pop_boost)

    print("Training algorithm: SVMKNN with sampling {}, similarity {} and pop_boost {}".format(sampling, similarity,
                                                                                                pop_boost))

    algo.fit(train)

    # save the model to disk
    filename = "models/valid/cb12_SVMKNN_Smpl" + str(sampling) + 'Sim' + str(similarity) + 'Pop_boost' + str(pop_boost) + ".model"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    print("Finished training. Storing model to: " + filename)
    with open(filename,'wb') as f:
        pickle.dump(algo,f)
    pass

In [ ]:
def trainGRU4Rec(loss, layers):

    algo = gru4rec.GRU4Rec(
        loss=loss,
        final_act='linear', 
        hidden_act='tanh', 
        layers=layers, 
        batch_size=32, 
        dropout_p_hidden=0.0, 
        learning_rate=0.2, 
        momentum=0.5, 
        n_sample=2048, 
        sample_alpha=0, 
        time_sort=True
    )

    layers_str = '_'.join(str(x) for x in layers)
    print("Training algorithm: GRU4Rec with loss {} and layers {}".format(loss, layers_str))

    algo.fit(train)

    # save the model to disk
    filename = "models/valid/cb12_GRU4Rec_loss" + loss + '_layers' + layers_str + ".model"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    print("Finished training. Storing model to: " + filename)
    with open(filename,'wb') as f:
        pickle.dump(algo,f)
    pass

In [ ]:
train_path = '../../data/cb12/processed/valid_train_14d.csv'
train = pd.read_csv(train_path, sep='\t')[['session_id', 'item_id', 'created_at']]
train.columns = ['SessionId', 'ItemId', 'Time']

In [ ]:
# Training BPR models
bpr_params = [[0, 0], [0, 0.25], [0, 0.5], [0.25, 0], [0.25, 0.25],[0.25, 0.5], [0.5, 0], [0.5, 0.25], [0.5, 0.5]]
for i in range (len(bpr_params)):
    trainBPR(bpr_params[i][0], bpr_params[i][1])

In [ ]:
# Training IKNN models

iknn_params = [[20, 0.25], [20, 0.5], [20, 0.75], [50, 0.25], [50, 0.5], [50, 0.75], [80, 0.25], [80, 0.5], [80, 0.75]]
for i in range(len(iknn_params)):
    trainIKNN(iknn_params[i][0], iknn_params[i][1])

In [ ]:
#Training sKNN

sknn_params = [["recent","jaccard",0],["recent","jaccard",1],
               ["recent", "cosine", 0],["recent","cosine",1],
               ["random","jaccard",0],["random","jaccard",1],
               ["random","cosine",0],["random","cosine",1]]

for i in range (len(sknn_params)):
    trainSKNN((sknn_params[i])[0],(sknn_params[i])[1],(sknn_params[i])[2])

In [ ]:
# Training SVKNN with same params as sknn
for i in range(len(sknn_params)):
    trainSVMKNN((sknn_params[i])[0],(sknn_params[i])[1],(sknn_params[i])[2])

In [ ]:
# Training S-sknn with same params as sknn

for i in range(len(sknn_params)):
    trainSsKNN((sknn_params[i])[0], (sknn_params[i])[1], (sknn_params[i])[2])

In [ ]:
# Training GRU4Rec models
gru_params = [
    ["top1-max", [100]], ["top1-max", [100,100]], ["top1-max", [1000]], ["top1-max", [1000,1000]], 
    ["bpr-max-0.5", [100]],["bpr-max-0.5", [100,100]], ["bpr-max-0.5", [1000]], ["bpr-max-0.5", [1000,1000]]
]
gru_params = [["bpr-max-0.5", [1000,1000]]]
for i in range (len(gru_params)):
    trainGRU4Rec(gru_params[i][0], gru_params[i][1])